In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
#path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
#path = "/data/sunilruf/prune/wanda/out/phi2_7b/model"
#path = "/data/sunilruf/phi/phi3/checkpoint-49/"
#path = "/data/sunilruf/phi/fp16_no/rank_32_1/checkpoint-50/"
path = "microsoft/Phi-3-mini-4k-instruct"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
rouge = evaluate.load('rouge')
import random
random.seed(0)

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

import torch
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    with torch.no_grad():
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        
        val = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return val, input_ids

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

In [11]:
import torch
from tqdm import tqdm
import numpy as np
import time
import json
start_time = time.time()

max_length = 1024
stride = 128
smoothie = SmoothingFunction().method4
#encodings = tokenizer.encode_plus(data['dialog'][1]['text'], return_tensors="pt").to('cuda')
tokenizer.pad_token = tokenizer.eos_token
seq_len = 1024


final_nlls = []

final_nlls_boh = []
prev_end_loc = 0
messages = []
i = 1
final_messages = []
final_messages_boh = []
final_messages_boh1 = []
final_perplexity = []
final_perplexity_boh = []
bleu_final, bleu_boh_final = [], []
rouge_1_final, rouge_2_final, rougle_l_final = [], [], []
Bert_P_final, Bert_R_final, Bert_F1_final = [], [], []
rouge_1_boh_final, rouge_2_boh_final, rougle_l_boh_final = [], [], []
Bert_P_boh_final, Bert_R_boh_final, Bert_F1_boh_final = [], [], []
output_len_final, output_len_boh_final = [], []
for j in range(10):
    #num = random.choice(range(3000, 3110))
    num = j
    #data = extes_data[num]['content']
    data = json.loads(dataset['test'][num]['text'])
    data = data['dialog']
    print(data)
    perplexity = []
    perplexity_boh = []
    nlls = []
    nlls_boh = []
    #data = data
    messages = []
    messages_boh = []
    messages_boh1 = []
    bleu_1, bleu_1_boh = [], []
    rouge_1, rouge_2, rougle_l = [], [], []
    Bert_P, Bert_R, Bert_F1 = [], [], []
    rouge_1_boh, rouge_2_boh, rougle_l_boh = [], [], []
    Bert_P_boh, Bert_R_boh, Bert_F1_boh = [], [], []
    output_len, output_len_boh = [], []
    try:
        for i in range(0, len(data), 1):
            for begin_loc in (range(0, seq_len, stride)):
                #print(data[i])
                
                if data[i]['speaker'] == 'sys':
                    break
                    
                #print(data[i]['User'])
                #print(data[i+1]['AI'])
                messages.append({"role": "user", "content":  data[i]['text']})
                print(messages)
                messages_boh.append({"role": "user", "content":  remove_stopwords(data[i]['text'])})
                messages_boh1.append({"role": "user", "content":  data[i]['text']})
                print(messages)
                #i+=1
                end_loc = min(begin_loc + max_length, seq_len)
                trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
                #input_ids = encodings.input_ids[:, begin_loc:end_loc].to('cuda')
                input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
                input_ids_boh = tokenizer.apply_chat_template(messages_boh, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
                target_ids = input_ids.clone()
                target_ids_boh = input_ids_boh.clone()
                target_ids[:, :-trg_len] = -100
                target_ids_boh[:, :-trg_len] = -100

                with torch.no_grad():
                    
                    
                    val, _ = generate_content(tokenizer, messages)
                    val_boh, _ = generate_content(tokenizer, messages_boh)
                    candidate = val
                    candidate_boh = val_boh
                    if data[i+1]['speaker'] == "sys":
                        reference = data[i+1]['text']
                    elif data[i+2]['speaker'] == "sys":
                        reference = data[i+1]['text']
                    elif data[i+3]['speaker'] == "sys":
                        reference = data[i+1]['text']
                        
                    references = reference.split(" ")
                    candidates = val.split(" ")
                    candidates_boh = val_boh.split(" ")
                    outputs = model(input_ids, labels=target_ids)
                    print(val)
                    print(val_boh)
                    messages.append({"role": "assistant", "content": val})
                    print(outputs.logits.shape)
                    output_len.append(outputs.logits.shape[1])
                    outputs_boh = model(input_ids_boh, labels=target_ids_boh)
                    print(outputs_boh.logits.shape)
                    output_len_boh.append(outputs_boh.logits.shape[1])
                    messages_boh.append({"role": "assistant", "content": remove_stopwords(val_boh)})
                    messages_boh1.append({"role": "assistant", "content": val_boh})
                        
                    
                    neg_log_likelihood = outputs.loss
                    neg_log_likelihood_boh = outputs_boh.loss
                    
                    bleu_1.append(sentence_bleu(references, candidates, weights=(1, 0, 0, 0), smoothing_function=smoothie))
                    results = rouge.compute(predictions=[candidate], references=[reference])
                    rouge_1.append(results['rouge1'])
                    rouge_2.append(results['rouge2'])
                    rougle_l.append(results['rougeL'])
                    
                    P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-tuned-uncased')
                    Bert_P.append(P)
                    Bert_R.append(R)
                    Bert_F1.append(F1)
                    
                    bleu_1_boh.append(sentence_bleu(references, candidates_boh, weights=(1, 0, 0, 0), smoothing_function=smoothie))
                    results_boh = rouge.compute(predictions=[candidate_boh], references=[reference])
                    rouge_1_boh.append(results_boh['rouge1'])
                    rouge_2_boh.append(results_boh['rouge2'])
                    rougle_l_boh.append(results_boh['rougeL'])
                    
                    P, R, F1 = score([candidate_boh], [reference], lang='en', model_type='bert-tuned-uncased')
                    Bert_P_boh.append(P)
                    Bert_R_boh.append(R)
                    Bert_F1_boh.append(F1)
                    #print(neg_log_likelihood.shape)

                nlls.append(neg_log_likelihood)
                nlls_boh.append(neg_log_likelihood_boh)
                #print(len(nlls))
                prev_end_loc = end_loc
                if end_loc == seq_len:
                    break

            ppl = torch.exp(torch.stack(nlls).mean())
            ppl_boh = torch.exp(torch.stack(nlls_boh).mean())
            print(ppl)
            print(ppl_boh)
            perplexity.append(ppl)
            perplexity_boh.append(ppl_boh)
            
    except Exception as e:
        print(e)
    
    final_perplexity.append(perplexity)
    final_perplexity_boh.append(perplexity_boh)
    final_nlls.append(nlls)
    final_nlls_boh.append(nlls_boh)
    final_messages.append(messages)
    final_messages_boh.append(messages_boh)
    final_messages_boh1.append(messages_boh1)
    bleu_final.append(bleu_1)
    rouge_1_final.append(rouge_1)
    rouge_2_final.append(rouge_2)
    rougle_l_final.append(rougle_l)
    Bert_P_final.append(Bert_P)
    Bert_R_final.append(Bert_R)
    Bert_F1_final.append(Bert_F1)
    bleu_boh_final.append(bleu_1_boh)
    rouge_1_boh_final.append(rouge_1_boh)
    rouge_2_boh_final.append(rouge_2_boh)
    rougle_l_boh_final.append(rougle_l_boh)
    Bert_P_boh_final.append(Bert_P_boh)
    Bert_R_boh_final.append(Bert_R_boh)
    Bert_F1_boh_final.append(Bert_F1_boh)
    output_len_final.append(output_len)
    output_len_boh_final.append(output_len_boh)

end_time = time.time()

[{'text': 'Hello. How are you today?', 'speaker': 'sys', 'strategy': 'Others'}, {'text': 'hi i am okay, a little bit sad though', 'speaker': 'usr'}, {'text': 'Okay. I am very sorry to hear that! Do you want to tell me more about that?', 'speaker': 'sys', 'strategy': 'Question'}, {'text': 'Well with the holidays coming up i have been very stressed and nervous about what i am going to do', 'speaker': 'usr'}, {'text': 'I really feel you there. Holidays are so hard.. especially with the way this year has been. Anything specific?', 'speaker': 'sys', 'strategy': 'Affirmation and Reassurance'}, {'text': "Yes. I am a college student and I live on campus. The day before I moved in my dad said some unforgivable things to me and we haven't spoke since", 'speaker': 'usr'}, {'text': 'I have no choice but to go live at home over the break but I am very scared about it', 'speaker': 'usr'}, {'text': "Do you feel scared for your own self? Or of the situation in general? I'm sorry to hear that. Relation

In [12]:
P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-tuned-uncased')

KeyError: 'bert-tuned-uncased'

In [9]:
print(end_time - start_time)

10.741971254348755


In [6]:
print(output_len_boh_final)
print(output_len_final)

[[], [], [], [5, 43, 171, 438, 983, 2113, 4096, 4096, 4096, 4096], [], [9, 68, 181, 459, 1044, 2213, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [5, 17, 37, 95, 242, 553, 1208, 2510, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [], [], [6, 19, 51, 142, 353, 828, 1781, 3667, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096]]
[[], [], [], [5, 76, 331, 793, 1788, 3772, 4096, 4096, 4096, 4096], [], [12, 157, 456, 1057, 2245, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [5, 24, 80, 234, 593, 1307, 2725, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [], [], [6, 23, 77, 263, 662, 1453, 3048, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096

In [7]:
def compute_average(value):
    value2 = np.array([])
    for val in value:
        #print(val)
        val = np.array(val)
        #print(val)
        #print(np.mean(val, axis=0))
        value2 = np.append(value2, np.mean(val, axis=0))
    #print(value2)
    return np.mean(value2)
        
def compute_bert(value):
    value2 = np.array([])
    for val in value:
        #print(val)
        val = np.array([tensor.item() for tensor in val])
        #print(val)
        #print(np.mean(val, axis=0))
        value2 = np.append(value2, np.mean(val, axis=0))
    #print(value2)
    return np.mean(value2)
def calc_out(value):
    try:
        value2 = []
        for i in value:
            value2.append(i[-1])
    except:
        pass
    return value2

In [8]:
import numpy as np

rouge1 = compute_average([x for x in rouge_1_final if x])
rouge2 = compute_average([x for x in rouge_2_final if x])
rougel = compute_average([x for x in rougle_l_final if x])
bleu = compute_average([x for x in bleu_final if x])
bert_p = compute_bert([x for x in Bert_P_final if x])
bert_r = compute_bert([x for x in Bert_R_final if x])
bert_f1 = compute_bert([x for x in Bert_F1_final if x])
perplexity = compute_bert([x for x in final_perplexity if x])
val1 = calc_out([x for x in output_len_final if x])
output_len = np.array(val1).mean()

rouge1_boh = compute_average([x for x in rouge_1_boh_final if x])
rouge2_boh = compute_average([x for x in rouge_2_boh_final if x])
rougel_boh = compute_average([x for x in rougle_l_boh_final if x])
bleu_boh = compute_average([x for x in bleu_boh_final if x])
bert_p_boh = compute_bert([x for x in Bert_P_boh_final if x])
bert_r_boh = compute_bert([x for x in Bert_R_boh_final if x])
bert_f1_boh = compute_bert([x for x in Bert_F1_boh_final if x])
perplexity_boh = compute_bert([x for x in final_perplexity_boh if x])
val2 = calc_out([x for x in output_len_boh_final if x])
output_len_boh = np.array(val2).mean()

print("Rouge1: ", rouge1)
print("Rouge2: ", rouge2)
print("RougeL: ", rougel)
print("Bleu: ", bleu)
print("Bert_P: ", bert_p)
print("Bert_R: ", bert_r)
print("Bert_F1: ", bert_f1)
print("Perplexity: ", perplexity)
print("Output Length: ", output_len)

print("Rouge1_boh: ", rouge1_boh)
print("Rouge2_boh: ", rouge2_boh)
print("RougeL_boh: ", rougel_boh)
print("Bleu_boh: ", bleu_boh)
print("Bert_P_boh: ", bert_p_boh)
print("Bert_R_boh: ", bert_r_boh)
print("Bert_F1_boh: ", bert_f1_boh)
print("Perplexity_boh: ", perplexity_boh)
print("Output Length_boh: ", output_len_boh)


Rouge1:  0.031929748663777464
Rouge2:  0.008478497278468236
RougeL:  0.027188889906260102
Bleu:  0.0017911843545493425
Bert_P:  0.3151673847470617
Bert_R:  0.4662717835074727
Bert_F1:  0.3736710604351599
Perplexity:  14.875036058612565
Output Length:  4096.0
Rouge1_boh:  0.03168344096547024
Rouge2_boh:  0.0040464483141907075
RougeL_boh:  0.026441112175996557
Bleu_boh:  0.0016541707548268039
Bert_P_boh:  0.2576054932737526
Bert_R_boh:  0.37963606418511264
Bert_F1_boh:  0.3044243390459341
Perplexity_boh:  176.79376011226356
Output Length_boh:  4096.0


In [9]:
print(final_perplexity)
print(final_perplexity_boh)

[[], [], [], [tensor(111.9675, device='cuda:0'), tensor(111.9675, device='cuda:0'), tensor(111.9675, device='cuda:0'), tensor(43.5904, device='cuda:0'), tensor(43.5904, device='cuda:0'), tensor(22.9601, device='cuda:0'), tensor(22.9601, device='cuda:0'), tensor(13.7238, device='cuda:0'), tensor(13.7238, device='cuda:0'), tensor(13.7238, device='cuda:0'), tensor(9.3396, device='cuda:0'), tensor(9.3396, device='cuda:0'), tensor(9.3396, device='cuda:0'), tensor(6.9031, device='cuda:0'), tensor(6.9031, device='cuda:0'), tensor(5.5362, device='cuda:0'), tensor(5.5362, device='cuda:0'), tensor(4.6917, device='cuda:0'), tensor(4.6917, device='cuda:0'), tensor(4.1251, device='cuda:0'), tensor(4.1251, device='cuda:0'), tensor(3.7215, device='cuda:0'), tensor(3.7215, device='cuda:0')], [], [tensor(131.7810, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(12.6060, device='cuda:0'), ten

In [10]:
final_perplexity = [x for x in final_perplexity if x]
final_perplexity_boh = [x for x in final_perplexity_boh if x]

print(final_perplexity)
print(final_perplexity_boh)
print(len(final_perplexity[0]))
print(len(final_perplexity_boh[2]))

[[tensor(111.9675, device='cuda:0'), tensor(111.9675, device='cuda:0'), tensor(111.9675, device='cuda:0'), tensor(43.5904, device='cuda:0'), tensor(43.5904, device='cuda:0'), tensor(22.9601, device='cuda:0'), tensor(22.9601, device='cuda:0'), tensor(13.7238, device='cuda:0'), tensor(13.7238, device='cuda:0'), tensor(13.7238, device='cuda:0'), tensor(9.3396, device='cuda:0'), tensor(9.3396, device='cuda:0'), tensor(9.3396, device='cuda:0'), tensor(6.9031, device='cuda:0'), tensor(6.9031, device='cuda:0'), tensor(5.5362, device='cuda:0'), tensor(5.5362, device='cuda:0'), tensor(4.6917, device='cuda:0'), tensor(4.6917, device='cuda:0'), tensor(4.1251, device='cuda:0'), tensor(4.1251, device='cuda:0'), tensor(3.7215, device='cuda:0'), tensor(3.7215, device='cuda:0')], [tensor(131.7810, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(25.6163, device='cuda:0'), tensor(12.6060, device='cuda:0'), tensor(8.1319, devi

In [11]:
for i in final_perplexity:
    for j in range(len(i)):
        i[j] = i[j].cpu()
        
for i in final_perplexity_boh:
    for j in range(len(i)):
        i[j] = i[j].cpu()

In [12]:
new_final_perplexity = []
new_final_perplexity_boh = []

for i in range(len(final_perplexity)):
    
    if len(final_perplexity[i]) > 20:
        new_final_perplexity.append(final_perplexity[i][:20])
        new_final_perplexity_boh.append(final_perplexity_boh[i][:20])
        

In [13]:
import numpy as np

perplexity_final_list_filtered = np.array(new_final_perplexity)
perplexity_boh_final_list_filtered = np.array(new_final_perplexity_boh)

print(np.mean(np.mean(perplexity_final_list_filtered, axis=1)))
print(np.mean(np.mean(perplexity_boh_final_list_filtered, axis=1)))

24.48886
399.1397


In [16]:
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/instruct-mistral/evaluate/perplexity_scores/esconv/"
np.save(path+'phi/phi_tuned.npy', perplexity_final_list_filtered)
np.save(path+'phi/phi_tuned_boh.npy', perplexity_boh_final_list_filtered)


In [17]:
print(output_len_final)
print(output_len_boh_final)

[[], [], [], [5, 76, 331, 793, 1788, 3772, 4096, 4096, 4096, 4096], [], [12, 157, 456, 1057, 2245, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [5, 24, 80, 234, 593, 1307, 2725, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [], [], [6, 23, 77, 263, 662, 1453, 3048, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096]]
[[], [], [], [5, 43, 171, 438, 983, 2113, 4096, 4096, 4096, 4096], [], [9, 68, 181, 459, 1044, 2213, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [5, 17, 37, 95, 242, 553, 1208, 2510, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096], [], [], [6, 19, 51, 142, 353, 828, 1781, 3667, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096, 4096

In [18]:
new_output_len = []
new_output_len_boh = []

for i in range(len(output_len_final)):
    
    if len(output_len_final[i]) > 5:
        new_output_len.append(output_len_final[i][:10])
        new_output_len_boh.append(output_len_boh_final[i][:10])

In [19]:
print(new_output_len)
print(new_output_len_boh)

[[5, 76, 331, 793, 1788, 3772, 4096, 4096, 4096, 4096], [12, 157, 456, 1057, 2245, 4096, 4096, 4096, 4096, 4096], [5, 24, 80, 234, 593, 1307, 2725, 4096, 4096, 4096], [6, 23, 77, 263, 662, 1453, 3048, 4096, 4096, 4096]]
[[5, 43, 171, 438, 983, 2113, 4096, 4096, 4096, 4096], [9, 68, 181, 459, 1044, 2213, 4096, 4096, 4096, 4096], [5, 17, 37, 95, 242, 553, 1208, 2510, 4096, 4096], [6, 19, 51, 142, 353, 828, 1781, 3667, 4096, 4096]]


In [20]:
import numpy as np

new_output_len_filtered = np.array(new_output_len)
new_output_len_boh_filtered = np.array(new_output_len_boh)

print(np.mean(np.mean(new_output_len_filtered, axis=1)))
print(np.mean(np.mean(new_output_len_boh_filtered, axis=1)))    

2065.8
1709.8249999999998


In [21]:

np.save(path+'phi/phi_tuned_out.npy', new_output_len_filtered)
np.save(path+'phi/phi_tuned_out_boh.npy', new_output_len_boh_filtered)
